In [1]:
from txgnn import TxData, TxGNN, TxEval

TxData = TxData(data_folder_path = '/n/scratch3/users/k/kh278/kg/')
TxData.prepare_split(split = 'complex_disease', seed = 42, no_kg = False)

Using backend: pytorch


Found local copy...
Downloading...


  0%|          | 0.00/8.89M [00:00<?, ?iB/s]

Done!
Downloading...


  0%|          | 0.00/387M [00:00<?, ?iB/s]

Done!
First time usage... Mapping RxData raw KG to directed csv... it takes several minutes...
Iterating over relations...


  0%|          | 0/30 [00:00<?, ?it/s]

Iterating over node types...


  0%|          | 0/10 [00:00<?, ?it/s]

Creating splits... it takes several minutes...
Creating DGL graph....
Done!


In [2]:
TxGNN = TxGNN(data = TxData, 
              weight_bias_track = False,
              proj_name = 'TxGNN',
              exp_name = 'TxGNN'
              )

# to load a pretrained model: 
# TxGNN.load_pretrained('./model_ckpt')

TxGNN.model_initialize(n_hid = 100, 
                      n_inp = 100, 
                      n_out = 100, 
                      proto = True,
                      proto_num = 3,
                      attention = False,
                      sim_measure = 'all_nodes_profile',
                      bert_measure = 'disease_name',
                      agg_measure = 'rarity',
                      num_walks = 200,
                      walk_mode = 'bit',
                      path_length = 2)

In [ ]:
## here we did not run this, since the output is too long to fit into the notebook
TxGNN.pretrain(n_epoch = 2, 
               learning_rate = 1e-3,
               batch_size = 1024, 
               train_print_per_n = 20)

In [5]:
## here as a demo, the n_epoch is set to 30. Change it to n_epoch = 500 when you use it
TxGNN.finetune(n_epoch = 30, 
               learning_rate = 5e-4,
               train_print_per_n = 5,
               valid_per_n = 20)

Epoch: 0 LR: 0.00050 Loss 0.6938, Train Micro AUROC 0.5109 Train Micro AUPRC 0.5127 Train Macro AUROC 0.5019 Train Macro AUPRC 0.5073
----- AUROC Performance in Each Relation -----
('drug', 'contraindication', 'disease'): 0.5349347764149285
('drug', 'indication', 'disease'): 0.5475017777377161
('drug', 'off-label use', 'disease'): 0.4750877230935641
('disease', 'rev_contraindication', 'drug'): 0.5200441594466088
('disease', 'rev_indication', 'drug'): 0.42414272145771154
('disease', 'rev_off-label use', 'drug'): 0.5098947539210383
----- AUPRC Performance in Each Relation -----
('drug', 'contraindication', 'disease'): 0.5439511651288017
('drug', 'indication', 'disease'): 0.551160435487897
('drug', 'off-label use', 'disease'): 0.4725924166335391
('disease', 'rev_contraindication', 'drug'): 0.5207788369454536
('disease', 'rev_indication', 'drug'): 0.4539515055664601
('disease', 'rev_off-label use', 'drug'): 0.5012481020151309
----------------------------------------------
Validation.....
E

In [6]:
TxGNN.save_model('./model_ckpt')
TxGNN.load_pretrained('./model_ckpt')

In [ ]:
TxGNN.train_graphmask(relation = 'indication',
                      learning_rate = 3e-4,
                      allowance = 0.005,
                      epochs_per_layer = 3,
                      penalty_scaling = 1,
                      epochs_per_layer = 5,
                      valid_per_n = 20)

output = TxGNN.retrieve_save_gates('./model_ckpt')
TxGNN.save_graphmask_model('./graphmask_model_ckpt')

In [9]:
from txgnn import TxEval
TxEval = TxEval(model = TxGNN)

In [4]:
# evaluate individual diseases
result = TxEval.eval_disease_centric(disease_idxs = [12661.0, 11318.0], 
                                     relation = 'indication', 
                                     save_result = False)

# evaluate the entire test set
result = RxEval.eval_disease_centric(disease_idxs = 'test_set', 
                                     show_plot = False, 
                                     verbose = True, 
                                     save_result = True,
                                     return_raw = False)

Evaluating relation: contraindication



---------
AUROC mean:  0.4827983690906781
AUROC std:  0.1710535011763153
---------
---------
AUPRC mean:  0.023629636234423025
AUPRC std:  0.036442407633213866
---------
---------
Accuracy mean:  0.974648837277499
Accuracy std:  0.04358800960119679
---------
---------
Sensitivity mean:  0.0
Sensitivity std:  0.0
---------
---------
Specificity mean:  1.0
Specificity std:  0.0
---------
---------
F1 mean:  0.0
F1 std:  0.0
---------
---------
PPV mean:  nan
PPV std:  nan
---------
---------
NPV mean:  0.974648837277499
NPV std:  0.04358800960119679
---------
---------
FPR mean:  0.0
FPR std:  0.0
---------
---------
FNR mean:  1.0
FNR std:  0.0
---------
---------
FDR mean:  nan
FDR std:  nan
---------
---------
# of Pos mean:  32.01851851851852
# of Pos std:  55.05165612631155
---------
---------
Recall@1% mean:  0.00129426532934482
Recall@1% std:  0.004735312292905053
---------
---------
Recall_Random@1% mean:  0.009496868416327983
Recall_Random@1% std:  0.0031843050687695726
-------


---------
AUROC mean:  0.43292636186911265
AUROC std:  0.21921846534384767
---------
---------
AUPRC mean:  0.006045431487876577
AUPRC std:  0.01036997622355921
---------
---------
Accuracy mean:  0.9958356468628541
Accuracy std:  0.006893993307483263
---------
---------
Sensitivity mean:  0.0
Sensitivity std:  0.0
---------
---------
Specificity mean:  1.0
Specificity std:  0.0
---------
---------
F1 mean:  0.0
F1 std:  0.0
---------
---------
PPV mean:  nan
PPV std:  nan
---------
---------
NPV mean:  0.9958356468628541
NPV std:  0.006893993307483263
---------
---------
FPR mean:  0.0
FPR std:  0.0
---------
---------
FNR mean:  1.0
FNR std:  0.0
---------
---------
FDR mean:  nan
FDR std:  nan
---------
---------
# of Pos mean:  7.5
# of Pos std:  12.41608194677737
---------
---------
Recall@1% mean:  0.0013331591779867642
Recall@1% std:  0.006319964413607548
---------
---------
Recall_Random@1% mean:  0.010273823015917559
Recall_Random@1% std:  0.0032496026468089674
---------
----


---------
AUROC mean:  0.41947627382973357
AUROC std:  0.2128356076699966
---------
---------
AUPRC mean:  0.010609126220454445
AUPRC std:  0.010198966578842047
---------
---------
Accuracy mean:  0.9931381248151433
Accuracy std:  0.005041650577382425
---------
---------
Sensitivity mean:  0.0
Sensitivity std:  0.0
---------
---------
Specificity mean:  1.0
Specificity std:  0.0
---------
---------
F1 mean:  0.0
F1 std:  0.0
---------
---------
PPV mean:  nan
PPV std:  nan
---------
---------
NPV mean:  0.9931381248151433
NPV std:  0.005041650577382425
---------
---------
FPR mean:  0.0
FPR std:  0.0
---------
---------
FNR mean:  1.0
FNR std:  0.0
---------
---------
FDR mean:  nan
FDR std:  nan
---------
---------
# of Pos mean:  3.3142857142857145
# of Pos std:  2.4351172288757157
---------
---------
Recall@1% mean:  0.0
Recall@1% std:  0.0
---------
---------
Recall_Random@1% mean:  0.008490884353741499
Recall_Random@1% std:  0.0020658633102465184
---------
---------
Enrichment@1%

In [10]:
TxEval.retrieve_disease_idxs_test_set('indication')

array([ 7701., 12661., 11195., 16569.,  6343.,  1479., 12621., 14416.,
       13557., 16501., 13446., 13610.,  7105.,  9716.,  5822., 12320.,
       10023., 14513., 12929.,  5683.,  9950., 14661.,  8395.,  7990.,
        6031.,  6255., 14409.,  9589., 11885.,  4285.,  6377., 13130.,
        7773.,  9741., 13381., 15355.,  4536., 12891.,  9087., 14873.,
        6185.,  1812., 15301.,  6474.,  3381.,  6599., 16965.,  7880.,
       16698.,  6252.,  5764.,  4192.,  4467., 13107., 14501., 11318.,
        3452., 11349., 15911., 15100.,  1417.,  4152.,  1319.,  2220.,
       15989.,  8973.])